In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
from typing import Union
from tqdm.auto import tqdm as tqdm
from sklearn import preprocessing
import gc
import lightgbm as lgb
import random
import os
import re 
import lightgbm as lgb
import dask.dataframe as dd
from sklearn import preprocessing, metrics
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit

In [2]:
data = pd.read_pickle('all_data_first.pkl')
sample_submission = pd.read_csv('sample_submission.csv')
data.shape

(22079892, 63)

Because of the size of our data, we will be using LGB to do our modeling and feature selection.

In [65]:
data.head()

,ATR,RS,ann_vol,beta,bollinger,cat_id,date,dayofweek,demand,dept_id,entropy,event,id,info_ratio,is_weekend,item_id,month,quarter,relative_vol,rolling_kurt_t30,rolling_max_14,rolling_max_180,rolling_max_30,rolling_max_60,rolling_max_90,rolling_mean_14,rolling_mean_180,rolling_mean_30,rolling_mean_60,rolling_mean_90,rolling_min_14,rolling_min_180,rolling_min_30,rolling_min_60,rolling_min_90,rolling_skew_t30,rolling_std_14,rolling_std_180,rolling_std_30,rolling_std_60,rolling_std_90,rrg_bench,rrg_item,sales_lag_15,sales_lag_17,sales_lag_19,sales_lag_21,sales_lag_23,sales_lag_25,sales_lag_27,sales_lag_29,sales_lag_31,sales_lag_33,sales_lag_35,sell_price,snap_CA,snap_TX,snap_WI,store_id,triple_exp,week,wm_yr_wk,year
0,3.0,0.0,18.234375,0.0,0.0,2,2014-05-14,2,0,5,1.559570,0.0,HOUSEHOLD_1_030_CA_4_validation,0.000865,0.0,2031,5,2.0,0.0,0.0,1.0,4.0,2.0,3.0,4.0,0.428467,0.688965,0.466553,0.566895,0.677734,0.0,0.0,0.0,0.0,0.0,0.0,0.513672,0.976074,0.628906,0.789062,0.897461,5.0,5.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,2.0,3.0,0.0,4.968750,0,0,1,3,0.120850,20,11415,2014
1,3.0,0.0,18.234375,0.0,2.0,2,2014-05-14,2,2,5,2.394531,0.0,HOUSEHOLD_1_032_CA_4_validation,0.021866,0.0,2032,5,2.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0,1.286133,0.655762,1.566406,1.383789,1.311523,0.0,0.0,0.0,0.0,0.0,0.0,1.772461,1.154297,1.612305,1.290039,1.346680,5.0,5.0,1.0,5.0,2.0,1.0,1.0,3.0,1.0,1.0,3.0,1.0,1.0,10.859375,0,0,1,3,1.757812,20,11415,2014
2,3.0,0.0,18.234375,0.0,0.0,2,2014-05-14,2,0,5,1.156250,0.0,HOUSEHOLD_1_033_CA_4_validation,-0.008560,0.0,2033,5,2.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.000000,0.183350,0.000000,0.133301,0.177734,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.441895,0.000000,0.389160,0.438965,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.468750,0,0,1,3,0.000000,20,11415,2014
3,3.0,0.0,18.234375,0.0,1.0,2,2014-05-14,2,1,5,1.230469,0.0,HOUSEHOLD_1_034_CA_4_validation,-0.007103,0.0,2034,5,2.0,0.0,0.0,2.0,5.0,5.0,5.0,5.0,0.285645,0.438965,0.566895,0.500000,0.444336,0.0,0.0,0.0,0.0,0.0,0.0,0.611328,0.799316,1.194336,0.947754,0.900879,5.0,5.0,0.0,0.0,1.0,0.0,4.0,1.0,0.0,5.0,0.0,0.0,0.0,3.970703,0,0,1,3,0.878906,20,11415,2014
4,3.0,0.0,18.234375,0.0,0.0,2,2014-05-14,2,0,5,0.443115,0.0,HOUSEHOLD_1_035_CA_4_validation,-0.018326,0.0,2035,5,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.005554,0.000000,0.000000,0.011108,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.074524,0.000000,0.000000,0.105408,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.628906,0,0,1,3,0.000000,20,11415,2014


In [3]:
# ca_model = data.loc[data['state_id']==0]
# tx_model = data.loc[data['state_id']==1]
# wi_model = data.loc[data['state_id']==2]

In [3]:
# data = pd.read_pickle('wi_model.pkl')

Running lgb on the entire dataset is too memory-intensive. So, we will first model the data by state to find feature importance. Contingent on the amount of features we drop, we hope that we can model the full data set.

In [4]:
# wi_items = '[A-Z]+_\d{1}\d?_\d+_Wi\w+'
# sample_submission = sample_submission.loc[sample_submission['id'].str.contains(wi_items)]
# sample_submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
12196,HOBBIES_1_001_TX_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12197,HOBBIES_1_002_TX_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12198,HOBBIES_1_003_TX_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12199,HOBBIES_1_004_TX_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12200,HOBBIES_1_005_TX_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
'''Columns we can drop due to no importance'''

to_drop = ['is_quarter_start','is_quarter_end', 'is_year_start',
           'gap_placement', 'gap_size', 'gap_start',
           'number_of_gaps', 'is_month_start', 'is_year_end', 'is_month_end']

data.drop(to_drop, axis=1, inplace=True)

KeyError: "['is_quarter_start' 'is_quarter_end' 'is_year_start' 'gap_placement'\n 'gap_size' 'gap_start' 'number_of_gaps' 'is_month_start' 'is_year_end'\n 'is_month_end'] not found in axis"

In [4]:
columns = [col for col in data.columns if col not in 
            ['id', 'demand', 'date', 'wm_yr_wk']]

The following lgb code was provided by: https://www.kaggle.com/ragnar123/asymmetric-loss-functions-lgbm

In [ ]:
def run_lgb(data, features, custom_asymmetric_train, custom_asymmetric_valid):
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-03-27']
    y_train = x_train['demand']
    x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    y_val = x_val['demand']
    test = data[(data['date'] > '2016-04-24')]
    del data
    gc.collect()

    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.1,
        'bagging_fraction': 0.75,
        'bagging_freq': 10, 
        'colsample_bytree': 0.75}

    train_set = lgb.Dataset(x_train[features], y_train)
    val_set = lgb.Dataset(x_val[features], y_val)
    
    del x_train, y_train

    model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, 
                      valid_sets = [train_set, val_set], verbose_eval = 100, fobj = custom_asymmetric_train, 
                      feval = custom_asymmetric_valid)
    
    val_pred = model.predict(x_val[features])
    y_pred = model.predict(test[features])
    x_val['demand'] = val_pred
    test['demand'] = y_pred
    x_val = x_val[['id', 'date', 'demand']]
    test = test[['id', 'date', 'demand']]

    return x_val, test

In [ ]:
def evaluate(x_val, train_fold_df, valid_fold_df, calendar, sell_prices):
    x_val = pd.pivot(x_val, index = 'id', columns = 'date', values = 'demand').reset_index()
    x_val.columns = ['id'] + ['d_' + str(i) for i in range(1886, 1914)]
    x_val = train_fold_df[['id']].merge(x_val, on = 'id')
    x_val.drop('id', axis = 1, inplace = True)
    evaluator = WRMSSEEvaluator(train_fold_df, valid_fold_df, calendar, sell_prices)
    score = evaluator.score(x_val)
    return score

def predict(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission.csv', index = False)

In [ ]:
# define cost and eval functions
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * 1.15)
    hess = np.where(residual < 0, 2, 2 * 1.15)
    return grad, hess

def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * 1.15) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [ ]:
# seed everything
seed_everything(42)
# reading data
data, calendar, sell_prices, submission, train_fold_df, valid_fold_df = read_data()
# get feature columns, also ignoring some features because we dont have enough ram and they overffit
features = [col for col in data.columns if col not in ['id', 'demand', 'part', 'date', 'wm_yr_wk', 'mean_demand_month', 'std_demand_month', 'max_demand_month', 'mean_demand_week', 'std_demand_week', 
                                                       'max_demand_week']]

print(f'We are training with {len(features)} fetures')
data.tail()

# run model with asymmetric loss function
x_val, test = run_lgb(data, features, custom_asymmetric_train, custom_asymmetric_valid)
score = evaluate(x_val, train_fold_df, valid_fold_df, calendar, sell_prices)
print(f'Our wrmsse is {score}')
# predict test
predict(test, submission)

Now that we have our predictions, it is time to add back our slope and intercepts since we stripped them for feature engineering and modeling.

In [15]:
final = pd.read_csv('submission.csv')
final.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.019411,0.014568,0.015195,0.014568,0.015195,0.016693,0.019411,0.018784,0.015195,0.014568,0.013374,0.013374,0.014871,0.016962,0.01759,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.016962
1,HOBBIES_1_002_CA_1_validation,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.018871,0.014568,0.014655,0.014568,0.014655,0.016153,0.018871,0.018784,0.014655,0.014568,0.012834,0.012834,0.014331,0.016962,0.01705,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.016962
2,HOBBIES_1_003_CA_1_validation,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.018871,0.014568,0.014655,0.014568,0.014655,0.016153,0.018871,0.018784,0.014655,0.014568,0.012834,0.012834,0.014331,0.016962,0.01705,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.016962
3,HOBBIES_1_004_CA_1_validation,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.018871,0.014568,0.014655,0.014568,0.014655,0.016153,0.018871,0.018784,0.014655,0.014568,0.012834,0.012834,0.014331,0.016962,0.01705,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.016962
4,HOBBIES_1_005_CA_1_validation,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.018871,0.014568,0.014655,0.014568,0.014655,0.016153,0.018871,0.018784,0.014655,0.014568,0.012834,0.012834,0.014331,0.016962,0.01705,0.012746,0.012746,0.012746,0.012746,0.014244,0.016962,0.016962


In [64]:
final.F1.value_counts()

0.000000    30490
0.012746    10191
0.013277     5415
0.012944     4366
0.013804     4201
0.013475     2303
0.014001     1806
0.012731     1093
0.012929      470
0.013788      230
0.013262      213
0.013460      109
0.013986       93
Name: F1, dtype: int64

In [16]:
sales = pd.read_csv('sales_train_validation.csv')
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [62]:
''' Grabbing the regression attributes from the original sales df
so we can add the slope back and scale back our submission values.'''

from scipy.stats import linregress

columns =['id','slope','intercept','std']
index = range(0, len(sales))
regress = pd.DataFrame(columns = columns, index = index)
regress.fillna(0, inplace=True)
d_cols = sales.columns[-712:]#since our data is only from d_1202 and beyond, we will only grab columns beyond those days.

for j, item in enumerate(sales.id.unique()):
    d_vals = np.squeeze(sales.loc[sales['id']==item][d_cols].T)
    item_len = range(0,len(d_vals))
    regress['id'].iloc[j] = item
    regress['slope'].iloc[j] = linregress(d_vals, item_len)[0]
    regress['intercept'].iloc[j] = linregress(d_vals, item_len)[1]
    regress['std'].iloc[j] = linregress(d_vals, item_len)[-1]

In [63]:
regress.head()

,id,slope,intercept,std
0,HOBBIES_1_001_CA_1_validation,24.833733,339.211582,8.431746
1,HOBBIES_1_002_CA_1_validation,-11.273509,359.315893,11.544826
2,HOBBIES_1_003_CA_1_validation,79.571820,327.225182,10.603538
3,HOBBIES_1_004_CA_1_validation,-8.532467,374.170764,3.337409
4,HOBBIES_1_005_CA_1_validation,11.664122,342.705507,6.167046


Now to repeat the same process for our predictions so we can scale properly.

In [85]:
for j, item in enumerate(loop_df['id'].unique()[:1]):
    print(item)

HOUSEHOLD_1_030_CA_4_validation


In [93]:
sales['d_1202'].dtype

dtype('int64')

In [99]:
data.loc[data['bollinger'].isna(), 'bollinger'] = data['demand']

In [ ]:
''' Grabbing the regression attributes from our predictions
so we can add the slope back and scale back our submission values.'''

columns =['id','slope','intercept','std']
index = range(0, len(loop_df))
final_reg = pd.DataFrame(columns = columns, index = index)
final_reg.fillna(0, inplace=True)

for j, item in enumerate(data['id'].unique()):
    d_vals = np.squeeze(data.loc[data['id']==item]['bollinger'])
    item_len = range(0,len(d_vals))
    final_reg['id'].iloc[j] = item
    final_reg['slope'].iloc[j] = linregress(d_vals, item_len)[0]
    final_reg['intercept'].iloc[j] = linregress(d_vals, item_len)[1]
    final_reg['std'].iloc[j] = linregress(d_vals, item_len)[-1]

In [ ]:
final_reg.head()